In [3]:
import pandas as pd
import numpy as np
import math
import os
import pickle
import collections
import json

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util import print_pd_table_for_libreoffice

/home/facade/projects/cgna/pose_estimation/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pkl_path = "~/mnt/remote1/checkpoints_all/baselines_1103_1108_Merge/test/comparison/test_results_analysis.pkl"
pkl_path = os.path.expanduser(pkl_path)

In [5]:
with open(pkl_path, 'rb') as f:
    d = pickle.load(f)

In [6]:
d.keys()

dict_keys(['mse', 'mse_desc', 'mpjre', 'mpjre_desc', 'mpjre_upper_body', 'mpjre_upper_body_desc', 'mpjre_lower_body', 'mpjre_lower_body_desc', 'mpjre_legs', 'mpjre_legs_desc', 'mpjpe', 'mpjpe_desc', 'mpjpe_upper_body', 'mpjpe_upper_body_desc', 'mpjpe_lower_body', 'mpjpe_lower_body_desc', 'mpjpe_legs', 'mpjpe_legs_desc', 'mpjve', 'mpjve_desc'])

In [7]:
d['mpjre_desc']

,insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe,insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe,ingpns+invel+inavel+mpjve+r6dn+mpjpe,ingp+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe,insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe,ingpn+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe,insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe
count,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000,94257.000000
mean,12.676493,15.043305,12.597822,12.140902,12.424694,12.731399,12.412940,12.528516,12.156551,12.681872,15.433396,12.764628,12.339309,12.665952
std,5.220784,6.078952,5.085363,5.020044,4.886660,5.224851,5.081106,5.267103,5.062103,5.279179,5.948586,5.169506,5.010492,5.196874
min,2.691182,3.166263,2.804009,2.776495,2.651531,2.770248,2.863062,3.087780,2.768014,3.049939,3.243220,2.934032,2.625721,3.300491
25%,9.179080,10.763503,9.179686,8.745947,9.110795,9.201760,9.044727,8.951601,8.791735,9.150790,11.137622,9.314222,8.905428,9.141847
50%,11.592069,13.696683,11.550942,11.118592,11.483993,11.650468,11.347970,11.444040,11.184365,11.593684,14.267611,11.701127,11.348376,11.630192
75%,14.654986,18.073825,14.609330,14.094876,14.410486,14.721876,14.403872,14.615903,14.124646,14.680195,18.706901,14.686712,14.381362,14.625174
max,50.606450,50.397720,43.605506,49.358712,48.193928,46.267406,48.928395,46.305635,46.477339,48.832777,48.115936,44.556781,45.771606,46.241706


In [8]:
d['mpjre_desc'].columns

Index(['insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe',
       'ingpns+invel+inavel+mpjve+r6dn+mpjpe',
       'ingp+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe',
       'ingpn+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe',
       'insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe'],
      dtype='object')

In [9]:
metrics = {
    'mpjpe': 'MPJPE [cm]',
    'mpjpe_legs': 'MPJPE Legs [cm]',
    'mpjve': 'MPJVE [cm/s]',
    'mpjre': 'MPJRE [°]',
    'mpjre_legs': 'MPJRE Legs [°]'
}
cmp_models = collections.defaultdict(list)

allmodelnames = d['mpjpe'].columns
hasvariations = '_' in allmodelnames[0]

for c in allmodelnames:
    if hasvariations:
        variations = c[:c.index("_")]
        modelc = c[c.index("_")+1:]
    else:
        variations = c
        modelc = 'default'
    cmp_models[modelc].append(variations)
    
print("cmp_models:\n", json.dumps(cmp_models, indent=2))

cmp_models:
 {
  "default": [
    "insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe"
  ]
}


In [10]:
l = list(cmp_models['default'])
cmp_models = collections.defaultdict(list)
for mc in l:
    m = 'mpjve' if 'mpjve' in mc else 'r6dn'
    cmp_models[m].append(mc)
print(json.dumps(cmp_models, indent=2))

{
  "mpjve": [
    "insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe"
  ]
}


In [11]:
# Reordering
cmp_models = {
    
  "nope": [
      "ingp+invel+inavel+mpjve+r6dn+mpjpe",
      "ingpn+invel+inavel+mpjve+r6dn+mpjpe",
      "ingpns+invel+inavel+mpjve+r6dn+mpjpe",
      
      "insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
  ],
  "pe": [
      "insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
      
      "insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe",
      
      "insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe",
    "insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe",
      "insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe",
  ]
}
    

def map_fn(met, val):
    if not met.startswith('mpjre'):
        val *= 100
    return val

In [12]:
dfs = {}
for modelc, variations in cmp_models.items():
    mc_data = collections.defaultdict(list)
    for v in variations:
        if hasvariations:
            fullname = v + "_" + modelc
        else:
            fullname = v
        for metric, metric_h in metrics.items():
            val = d[metric + "_desc"].loc['mean', fullname]
            val = map_fn(metric, val)
            mc_data[fullname].append(val)
    df = pd.DataFrame(data=mc_data, index=metrics.values())
    dfs[modelc] = df
    

In [13]:
dfs.keys()

dict_keys(['nope', 'pe'])

In [14]:
pd.options.display.float_format = '{:.2f}'.format

In [15]:
# PASTE SPECIAL -> UNFORMATTED TEXT
_ = print_pd_table_for_libreoffice(dfs['nope'])

	ingp+invel+inavel+mpjve+r6dn+mpjpe	ingpn+invel+inavel+mpjve+r6dn+mpjpe	ingpns+invel+inavel+mpjve+r6dn+mpjpe	insmp+ingp+invel+inavel+mpjve+r6dn+mpjpe	insmp+ingpn+invel+inavel+mpjve+r6dn+mpjpe	insmp+ingpnys+invel+inavel+mpjve+r6dn+mpjpe
MPJPE	[cm]	7.44	7.43	7.43	7.49	7.48	9.86
MPJPE	Legs	[cm]	9.34	9.32	9.22	9.37	9.37	12.66
MPJVE	[cm/s]	19.16	18.38	18.76	18.76	18.74	23.37
MPJRE	[°]	12.42	12.16	12.14	12.41	12.34	15.04
MPJRE	Legs	[°]	14.06	13.76	13.88	13.83	13.88	18.08


In [16]:
# PASTE SPECIAL -> UNFORMATTED TEXT
_ = print_pd_table_for_libreoffice(dfs['pe']) 

	insmp+pea1+ingp+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea1+ingpns+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea2+ingp+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea2+ingpn+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea2+ingpnys+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea4+ingp+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea4+ingpn+invel+inavel+mpjve+r6dn+mpjpe	insmp+pea4+ingpns+invel+inavel+mpjve+r6dn+mpjpe
MPJPE	[cm]	7.54	7.43	7.61	7.46	10.26	7.59	7.51	7.50
MPJPE	Legs	[cm]	9.33	9.18	9.34	9.20	13.71	9.29	9.22	9.17
MPJVE	[cm/s]	19.07	18.60	18.67	18.66	23.45	18.75	18.56	18.84
MPJRE	[°]	12.53	12.60	12.67	12.68	15.43	12.68	12.76	12.73
MPJRE	Legs	[°]	13.96	13.91	13.96	13.86	18.66	13.90	13.96	13.84


In [25]:
_ = print_pd_table_for_libreoffice(dfs['default'])

KeyError: 'default'

In [26]:
dfs['mpjve']

,ingp+invel+inavel+mpjve+r6dn+mpjpe,ingpn+invel+inavel+mpjve+r6dn+mpjpe,ingpns+invel+inavel+mpjve+r6dn+mpjpe,ingpns+invel+mpjve+r6dn+mpjpe,ingpns+mpjve+r6dn+mpjpe,ingpnys+invel+inavel+mpjve+r6dn+mpjpe,ingpnys+invel+mpjve+r6dn+mpjpe,ingpnys+mpjve+r6dn+mpjpe,insmp+pea1+ingpnys+invel+inavel+mpjve+r6dn+mpjpe,insmp+pea1+ingpn+invel+inavel+mpjve+r6dn+mpjpe
MPJPE [cm],7.42,7.54,7.49,8.12,8.19,9.51,10.13,10.02,9.65,7.69
MPJPE Legs [cm],9.28,9.30,9.30,9.66,9.72,11.68,13.21,12.72,12.03,9.45
MPJVE [cm/s],18.68,18.70,18.77,20.72,21.33,23.01,23.91,24.02,23.42,19.07
MPJRE [°],12.16,12.14,12.13,12.87,12.91,14.57,15.11,14.97,14.92,12.83
MPJRE Legs [°],13.72,13.82,13.79,13.99,14.18,17.14,18.58,17.97,17.50,14.10
